In [1]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import torch
import pickle
import glob
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas
import time
import os

import connect_four
import mcts
import connect_net
import connect_plot
import evaluation
import utils
import runner
%matplotlib inline
%load_ext autoreload
%autoreload 2
os.chdir("/Users/raymond/code/FinalProject563")

In [3]:
data_folder = utils.get_data_folder()
boards = np.load(os.path.join(data_folder, "move8_boards.npy"))
winners = np.load(os.path.join(data_folder, "move8_winner.npy"))
boards = torch.Tensor(boards)
winners = torch.Tensor(winners)
ones = torch.ones_like(winners)

In [8]:
iteration = 63
trial = "test01"
training_folder = utils.get_training_folder(trial=trial, iteration=iteration)
training_path = runner.get_model_path(folder=training_folder, iteration=iteration)
nnet63 = connect_net.load_model(path=training_path, log=False)

In [13]:
v, proba = nnet63(boards.view(-1,1,6,7))

In [17]:
winners[:10]

tensor([1., 1., 1., 1., 1., 1., 0., 1., 1., 1.])

In [20]:
max_v = torch.where(v > 0, torch.ones_like(v), torch.zeros_like(v))
max_v[:10]

tensor([1., 1., 0., 1., 1., 1., 1., 1., 1., 1.])

In [58]:
37924 / (37924 + 29633)

0.5613629971727578

In [23]:
pandas.Series((max_v == winners).tolist()).value_counts()

True     37924
False    29633
dtype: int64

In [39]:
game_cache = connect_four.GameCache()
strategy = evaluation.NetworkStrategy(nnet=nnet63, c=1, num_mcts_sims=32, random_moves=0)
strategy.setup(game_cache=game_cache)

In [59]:
vals = []
vals_alt = []
for i, board in enumerate(boards):
    if i % 1000 == 999:
        print(i)
        break
    rep = connect_four.to_rep(board)
    strategy.simulate(state=board, rep=rep)
    vals.append(strategy.get_best_eval(rep=rep))
    vals_alt.append(strategy.get_eval(rep=rep).max())
        
vals = torch.stack(vals)
vals_alt = torch.stack(vals_alt)

999


In [61]:
vals_ones = torch.ones_like(vals)
max_v = torch.where(vals.abs() < .1, torch.zeros_like(vals), torch.where(vals > 0, vals_ones, -vals_ones))
max_v_alt = torch.where(vals_alt.abs() < .1, torch.zeros_like(vals), torch.where(vals_alt > 0, vals_ones, -vals_ones))

In [62]:
pandas.Series((max_v == winners[:999]).tolist()).value_counts()

True     785
False    214
dtype: int64

In [63]:
pandas.Series((max_v_alt == winners[:999]).tolist()).value_counts()

True     785
False    214
dtype: int64